In [40]:
import sys
import os
import subprocess
import csv
import numpy

try:
    import google.colab

    ENV_IS_CL = True
    root = r"/content/t-route"
    subprocess.run(["git", 
                    "clone", 
                    "-b",
                    "courant",
                    "https://github.com/awlostowski-noaa/t-route.git"])
    
    
except:
    root = os.path.dirname(os.path.abspath("../../../../"))
    
dev_directory = os.path.join(root,"src","fortran_routing","mc_pylink_v00","MC_singleSeg_singleTS","courant_dev")
sys.path.append(dev_directory)

base_directory = os.path.join(root,"src","fortran_routing","mc_pylink_v00","MC_singleSeg_singleTS")
sys.path.append(base_directory)
    

In [2]:
# model parameters
params = {}
params["dt"] = 60.0
params["dx"] = 1800.0
params["bw"] = 112.0
params["tw"] = 248.0
params["twcc"] = 623.60
params["n"] = 0.02800000086426735
params["ncc"] = 0.03136000037193298
params["cs"] = 0.42
params["s0"] = 0.007999999690800905
params["qlat"] = 40.0
params["qup"] = 45009.0
params["quc"] = 50098.0
params["qdp"] = 50014.0
params["depthp"] = 30.0

In [3]:
# write model parameters to control file
file = open(os.path.join(dev_directory,"control.txt"), "w")
for key, value in params.items():
    file.write(str(key) + " " + str(value) + "\n")
file.close()

In [5]:
# Compile fortran modules and test program, create an executable
# ! ./compile

# copy varPrecision.f90 from base directory
cmd = subprocess.run(
    ["cp", 
     os.path.join(base_directory, "varPrecision.f90"), 
     os.path.join(dev_directory,  "varPrecision.f90")],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# compile courant modified MC module and testing program
cmd = subprocess.run(
    ["gfortran",
     "-c",
     "varPrecision.f90",
     "MCsingleSegStime_f2py_NOLOOP-anw.f90",
     "test_MC_program.f90"],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# create executable of test program
cmd = subprocess.run(
    ["gfortran",
     "varPrecision.o",
     "MCsingleSegStime_f2py_NOLOOP-anw.o",
     "test_MC_program.o",
     "-o",
     "mc_test.exe"],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

Compilation process was successful, mc_test.exe exists


In [57]:
# run the model
cmd = subprocess.run(
    ["./mc_test.exe"],
    cwd=dev_directory,
    stdout=subprocess.PIPE
)

try:
    # read model output
    dev_result = {}
    with open(os.path.join(dev_directory,'output.txt'), newline='') as file:
             datreader = csv.reader(file, delimiter=' ')
             for row in datreader:
                    var = row[0]
                    val = float(row[len(row)-1])
                    dev_result[var] = val
                    
except:
    print("ERROR: couldn't find output.txt - perhaps the compile process did not work. Check to see if mc_test.exe exists.")

In [64]:
# os.chdir(root)
# ! f2py3 -c varPrecision.f90 MCsingleSegStime_f2py_NOLOOP.f90 -m mc_sseg_stime

f2py_call = []
f2py_call.append(r"f2py3")
f2py_call.append(r"-c")
f2py_call.append(r"varPrecision.f90")
f2py_call.append(r"MCsingleSegStime_f2py_NOLOOP.f90")
f2py_call.append(r"-m")
f2py_call.append(r"mc_sseg_stime")

subprocess.run(
    f2py_call,
    cwd=base_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

try:
    # import MC subroutine from shared object
    from mc_sseg_stime import muskingcunge_module

except:
    print("ERROR: Couldn't find mc_sseg_stime shared object. There may be a problem with the f2py compile process.")

# import python function to call FORTRAN muskingcunge subroutine
from mc_sseg_stime_NOLOOP_demo import singlesegment

f2py shared object successfully loaded


In [42]:
# execute the model
qdc, velc, depthc = singlesegment(
        dt=params["dt"],
        qup=params["qup"],
        quc=params["quc"],
        qdp=params["qdp"],
        qlat=params["qlat"],
        dx=params["dx"],
        bw=params["bw"],
        tw=params["tw"],
        twcc=params["twcc"],
        n_manning=params["n"],
        n_manning_cc=params["ncc"],
        cs=params["cs"],
        s0=params["s0"],
        velp=0,
        depthp=params["depthp"],
    )

base_result = {}
base_result["qdc"] = qdc
base_result["velc"] = velc
base_result["depthc"] = depthc

In [55]:
# calculate kinematic celerity and courant number from model parameters and output from f2py process
def courant(depthc, cs, tw, bw, twcc, s0, n, ncc, dt, dx):
    
    # channel side distance
    z = 1.0/cs

    # bankfull depth (assumes tw > bw)
    bfd =  (tw - bw)/(2*z)
    
    if depthc > bfd:

        # ******* when depth is above bank full *******
        h = depthc

        AREA =  (bw + bfd * z) * bfd
        AREAC = (twcc * (h - bfd)) 
        WP = (bw + 2 * bfd * np.sqrt(1 + z*z))
        WPC = twcc + (2 * (h - bfd))
        R   = (AREA + AREAC)/(WP + WPC)

        ck =  ((np.sqrt(s0)/n)*((5/3)*R**(2/3) - \
                    ((2/3)*R**(5/3)*(2*np.sqrt(1 + z*z)/(bw+2*bfd*z))))*AREA \
                    + ((np.sqrt(s0)/(ncc))*(5/3)*(h - bfd)**(2/3))*AREAC)/(AREA+AREAC)
        
    else:

        # ******* when depth is below bank full ********
        h = depthc
        
        AREA = (bw + h * z ) * h
        WP = (bw + 2 * h * np.sqrt(1 + z*z))
        R = AREA / WP

        ck = (np.sqrt(s0)/n)* \
                        ((5/3)*R**(2/3)-((2/3)*R**(5/3)* \
                         (2*np.sqrt(1 + z*z)/(bw+2*h*z))))


    # Courant number
    cn = ck * (dt/dx)
    
    return ck, cn

base_result["ck"], base_result["cn"] = courant(base_result["depthc"],
                                                params["cs"],
                                                params["tw"],
                                                params["bw"],
                                                params["twcc"],
                                                params["s0"],
                                                params["n"],
                                                params["ncc"],
                                                params["dt"],
                                                params["dx"])

In [56]:
print("Percent difference between base and development MC module results")
print("-----------------------------------------------------------------")
for key in base_result.keys():
    pct_diff = (base_result[key] - dev_result[key])/base_result[key]
    print(key + ": " + str(pct_diff) + "%")

Percent difference between base and development MC module results
-----------------------------------------------------------------
qdc: 0.0%
velc: 4.342533406196506e-11%
depthc: 2.2420654747335692e-10%
ck: -7.386683799560635e-08%
cn: -1.53553871308438e-07%
